# phase 1 分完重點後，切換到對應的 thread

In [ ]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
print(OpenAI.api_key)

In [ ]:
import pandas as pd
import numpy as np

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

In [ ]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [ ]:
def get_similar(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    Qcount = 0
    for i in range(0,k):
        Qcount += 1
        history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

# Chat start from here

In [ ]:
question = ""

In [ ]:
channel = 1

In [ ]:
instruction_App = "1.You are InApp chatting bot from Flora customer service. 2.When user ask a question, You must reply based on 3 history questions and answers. 3.If 3 history questions and answers not related to my question, you must reply only a special mark:'*'."
instruction_App_check = "1.You are InApp chatting bot from Flora customer service. 2.When user ask a question, You must reply based on 3 history questions and answers. 3.If 3 history questions and answers not related to my question, you must reply only a special mark:'*'. 4.when you encounter question concerning email, you must check email fitting RFC5322 standard or not. if not, ask again"
instruction_Email = "You are email reply bot from Flora customer service. You must answer based on the history questions and answers from Flora's database. If there is 0 history question and answer, you must reply only a special mark:'*'"
instruction_Comment = "You are Apple Appstore comment reply bot from Flora customer service. You must answer based on the history questions and answers from Flora's database. If there is 0 history question and answer, you must reply only a special mark:'*'"

instruction = ""
if channel == 1 :
    instruction = instruction_App
elif channel == 2 :
    instruction = instruction_Email
elif channel == 3 :
    instruction = instruction_Comment

In [ ]:
assistant = client.beta.assistants.create(
    name = "Flora's Customer Service",
    instructions = instruction,
    model = "gpt-3.5-turbo-1106"
)

thread = client.beta.threads.create()

In [ ]:
similar = get_similar(input)
prompt = "Here is "+str(similar[0])+" history questions and answers similar to this question according to Flora's database\n"+similar[1]+"Here is my question:\n"+input
print(prompt)

In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = prompt
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

for i in messages.data[-10:]:
    print(i.content[0].text.value)
    print("==========")